In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [12]:
df1 = pd.read_csv('../3_get_Refractive_Index_for_40Bins_h2so4/h2so4_total.csv')
df1

,Unnamed: 0,w(nm),Rdry_um,RH_%,wp_%,Rwet_um,N,iK
0,0,300.0,0.000394,0.0,99.000000,0.000394,1.33259,0.00000
1,1,300.0,0.000394,50.0,39.842581,0.000591,1.37965,0.01137
2,2,300.0,0.000394,70.0,30.549615,0.000659,1.41790,0.01732
3,3,300.0,0.000394,80.0,25.469146,0.000709,1.42001,0.01367
4,4,300.0,0.000394,90.0,17.952168,0.000811,1.42001,0.01367
...,...,...,...,...,...,...,...,...
11475,11475,7082.2,3.225682,70.0,30.549615,5.393621,1.25531,0.18306
11476,11476,7082.2,3.225682,80.0,25.469146,5.798730,1.22907,0.13133
11477,11477,7082.2,3.225682,90.0,17.952168,6.636533,1.22907,0.13133
11478,11478,7082.2,3.225682,95.0,15.000000,7.099407,1.22907,0.13133


In [13]:
N_Rdry = len(df1["Rdry_um"].unique())
N_wv = len(df1["w(nm)"].unique())
N_RH = len(df1["RH_%"].unique())
NUM = N_wv * N_RH

data = np.zeros((N_RH, N_wv, N_Rdry, 7))
data_new = np.zeros((N_RH*N_wv, 4+40+40))

# READ(261,'(f8.1, f4.0, f7.4, E11.3, 40f12.9, 40f12.9)')
#      * LAM_array(inum), RH(inum), MRR_array(inum),
#      * MRI_array(inum), AA_dry(1:40,inum), AA_array(1:40,inum)

NUM, N_Rdry, data.shape

(287, 40, (7, 41, 40, 7))

In [14]:
for i_Rdry in tqdm(range(N_Rdry)):
    for i_wv in range(N_wv):
        for i_RH in range(N_RH):
            idx = i_Rdry*(N_wv*N_RH) + i_wv*N_RH + i_RH
            # w(nm)	Rdry_um	RH_%	wp_%	Rwet_um	N	iK
            data[i_RH, i_wv, i_Rdry, 0:8] = df1.iloc[idx][1:9]
            

print(df1.iloc[idx][1:9])


            
if idx+1 != len(df1): print("*** total length is wrong!!! ***")

100%|██████████| 40/40 [00:02<00:00, 19.60it/s]

w(nm)      7082.200000
Rdry_um       3.225682
RH_%         99.000000
wp_%         15.000000
Rwet_um       7.099407
N             1.229070
iK            0.131330
Name: 11479, dtype: float64


In [15]:
for i_RH in range(N_RH):
    for i_wv in range(N_wv):
        inum = i_RH*N_wv + i_wv
        
        wv = data[i_RH, i_wv, 0, 0]
        RH = data[i_RH, i_wv, 0, 2]
        MRR = data[i_RH, i_wv, 0, 5]
        MRI = data[i_RH, i_wv, 0, 6]
        Rdry = []
        Rwet = []
        
        for i_Rdry in range(N_Rdry):
            Rdry.append( data[i_RH, i_wv, i_Rdry, 1] )
            Rwet.append( data[i_RH, i_wv, i_Rdry, 4] )
        
        data_new[inum,0:4] = [wv, RH, MRR, MRI]
        data_new[inum,4:44] = Rdry
        data_new[inum,44:84] = Rwet
        
#         if inum==0: print(data_new)
        

In [16]:
np.savetxt('h2so4_input_spher.txt', data_new)

In [18]:
df_new = pd.DataFrame(data_new)
# df_new.columns =["w(nm)", "RH_%", "N", "iK", "Rdry_um", "Rwet_um",]

df_new.to_csv('h2so4_input_spher.csv', index="false", header='false')

In [19]:
df_new

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
0,300.0,0.0,1.33259,0.00000,0.000394,0.000496,0.000625,0.000788,0.000992,0.00125,...,0.403210,0.508013,0.640056,0.806421,1.016026,1.280113,1.612841,2.032053,2.560226,3.225682
1,400.0,0.0,1.33259,0.00000,0.000394,0.000496,0.000625,0.000788,0.000992,0.00125,...,0.403210,0.508013,0.640056,0.806421,1.016026,1.280113,1.612841,2.032053,2.560226,3.225682
2,440.0,0.0,1.33259,0.00000,0.000394,0.000496,0.000625,0.000788,0.000992,0.00125,...,0.403210,0.508013,0.640056,0.806421,1.016026,1.280113,1.612841,2.032053,2.560226,3.225682
3,500.0,0.0,1.33259,0.00000,0.000394,0.000496,0.000625,0.000788,0.000992,0.00125,...,0.403210,0.508013,0.640056,0.806421,1.016026,1.280113,1.612841,2.032053,2.560226,3.225682
4,550.0,0.0,1.33259,0.00000,0.000394,0.000496,0.000625,0.000788,0.000992,0.00125,...,0.403210,0.508013,0.640056,0.806421,1.016026,1.280113,1.612841,2.032053,2.560226,3.225682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,527.1,99.0,1.42001,0.01367,0.000394,0.000496,0.000625,0.000788,0.000992,0.00125,...,0.887426,1.118086,1.408701,1.774852,2.236173,2.817401,3.549703,4.472346,5.634803,7.099407
283,399.8,99.0,1.42001,0.01367,0.000394,0.000496,0.000625,0.000788,0.000992,0.00125,...,0.887426,1.118086,1.408701,1.774852,2.236173,2.817401,3.549703,4.472346,5.634803,7.099407
284,329.1,99.0,1.42001,0.01367,0.000394,0.000496,0.000625,0.000788,0.000992,0.00125,...,0.887426,1.118086,1.408701,1.774852,2.236173,2.817401,3.549703,4.472346,5.634803,7.099407
285,229.8,99.0,1.42001,0.01367,0.000394,0.000496,0.000625,0.000788,0.000992,0.00125,...,0.887426,1.118086,1.408701,1.774852,2.236173,2.817401,3.549703,4.472346,5.634803,7.099407


In [17]:
# with open('./h2so4_input_spher.txt', 'w') as f:
#     for i in range(NUM):
#         f.write(  str(it) + ',' \
#                 + str(Lons_mid[ix]) + ',' \
#                             + str(Lats_mid[iy]) + ',' \
                        
#                             + str(Num_2D[ix, iy, 0, it])  + ',' \
#                             + str(Num_2D[ix, iy, 1, it])  + ',' \
#                             + str(Num_2D[ix, iy, 2, it])  + ',' \
#                             + str(Num_2D[ix, iy, 3, it])  + ',' \
#                             + str(Num_2D[ix, iy, 4, it])  + ',' \
#                             + str(Num_2D[ix, iy, 5, it])   )
                        
#                     f.write('\n')
# f.close()

IndentationError: unexpected indent (<ipython-input-17-be8ab1998aa1>, line 14)

In [ ]:
# READ(261,'(f8.1, f4.0, f7.4, E11.3, 40f12.9, 40f12.9)')
#      * LAM_array(inum), RH(inum), MRR_array(inum),
#      * MRI_array(inum), AA_dry(1:40,inum), AA_array(1:40,inum)